In [ ]:
!pip install pandas numpy scikit-learn openpyxl nltk contractions
import pandas as pd
import numpy as np
import re
import nltk
import contractions

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

import joblib
nltk.download('punkt')
nltk.download('stopwords')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 5.5 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving agents_dedup_and_cleaned_genres .xlsx to agents_dedup_and_cleaned_genres .xlsx


In [ ]:
df = pd.read_excel(list(uploaded.keys())[0])
df.head()


,Agent Name,Agency,Country,Sources (URLs only),Notable Clients / Titles,Genres / Categories (Cleaned)
0,A.J. Van Belle,The Booker Albert Literary Agency,USA,https://www.thebookeralbertagency.com/,NaN,"Romance, Contemporary, Rom-com"
1,Aashna Avachat,Neighborhood Literary,USA,https://neighborhoodlit.com/,NaN,"Young Adult, Commercial, Romance, Sff"
2,Abby Saul,The Lark Group,USA,https://thelarkwords.com/,NaN,"Upmarket, Commercial Fiction, Suspense, Women S"
3,Abi Fellows,Dhh Literary,United Kingdom,https://www.dhhliteraryagency.com/,NaN,"Literary, Upmarket, Commercial Fiction, Young ..."
4,Abigail Koons,"Park, Fine & Brower Literary Management",USA,https://parkfine.com/our-team/,NaN,"Upmarket, Commercial Fiction, Narrative Nonfic..."


In [ ]:
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
df.columns


Index(['agent_name', 'agency', 'country', 'sources_(urls_only)',
       'notable_clients_/_titles', 'genres_/_categories_(cleaned)'],
      dtype='object')

In [ ]:
df = df[["agent_name", "agency", "country", "genres_/_categories_(cleaned)"]]
df.head()


,agent_name,agency,country,genres_/_categories_(cleaned)
0,A.J. Van Belle,The Booker Albert Literary Agency,USA,"Romance, Contemporary, Rom-com"
1,Aashna Avachat,Neighborhood Literary,USA,"Young Adult, Commercial, Romance, Sff"
2,Abby Saul,The Lark Group,USA,"Upmarket, Commercial Fiction, Suspense, Women S"
3,Abi Fellows,Dhh Literary,United Kingdom,"Literary, Upmarket, Commercial Fiction, Young ..."
4,Abigail Koons,"Park, Fine & Brower Literary Management",USA,"Upmarket, Commercial Fiction, Narrative Nonfic..."


In [ ]:
df["genres_/_categories_(cleaned)"] = df["genres_/_categories_(cleaned)"].fillna("")
df["country"] = df["country"].fillna("").str.lower()
df["agency"] = df["agency"].fillna("").str.lower()


In [ ]:
training_rows = []

for i, agent in df.iterrows():
    genres = str(agent["genres_/_categories_(cleaned)"]).lower()

    # Positive example (1)
    training_rows.append({
        "input_genre": genres,
        "label": agent["agent_name"]
    })

    # Negative example (0)
    training_rows.append({
        "input_genre": "unrelated romance thriller wizard vampire",  # random negative
        "label": agent["agent_name"]
    })

train_df = pd.DataFrame(training_rows)
train_df.head()


,input_genre,label
0,"romance, contemporary, rom-com",A.J. Van Belle
1,unrelated romance thriller wizard vampire,A.J. Van Belle
2,"young adult, commercial, romance, sff",Aashna Avachat
3,unrelated romance thriller wizard vampire,Aashna Avachat
4,"upmarket, commercial fiction, suspense, women s",Abby Saul


In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(train_df["input_genre"])


In [ ]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(train_df["label"])


In [ ]:
rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=40,
    class_weight="balanced",
    random_state=42
)

rf.fit(X, y)


RandomForestClassifier(class_weight='balanced', max_depth=40, n_estimators=300,
                       random_state=42)

In [ ]:
joblib.dump(rf, "agent_rf_model.pkl")
joblib.dump(vectorizer, "agent_vectorizer.pkl")
joblib.dump(label_encoder, "agent_label_encoder.pkl")


['agent_label_encoder.pkl']